In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
attacks_original = pd.read_csv('../data/attacks.csv', encoding = 'latin1')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

attacks = attacks_original.copy()

In [3]:
attacks_original.shape

(25723, 24)

In [4]:
attacks.columns = attacks.columns.str.strip().str.lower().str.replace(' ','_').str.replace('.','_').str.replace(':','')

In [5]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number                66.17
date                       75.50
year                       75.51
type                       75.52
country                    75.69
area                       77.27
location                   77.60
activity                   77.62
name                       76.32
sex                        77.70
age                        86.51
injury                     75.61
fatal_(y/n)                77.60
time                       88.54
species                    86.53
investigator_or_source     75.57
pdf                        75.50
href_formula               75.50
href                       75.50
case_number_1              75.50
case_number_2              75.50
original_order             75.47
unnamed_22                100.00
unnamed_23                 99.99
dtype: float64

In [6]:
attacks = attacks.dropna(thresh = 3) #de esta manera nos quedamos con las filas que al menos tienen 3 columnas con datos.
attacks.shape

(6302, 24)

In [7]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number                0.02
year                       0.03
type                       0.06
country                    0.79
area                       7.22
location                   8.57
activity                   8.63
name                       3.33
sex                        8.97
age                       44.92
injury                     0.44
fatal_(y/n)                8.55
time                      53.22
species                   45.03
investigator_or_source     0.27
href_formula               0.02
unnamed_22                99.98
unnamed_23                99.97
dtype: float64

In [8]:
attacks.fillna({'type':'unknown', 'location' :  'unknown','name':'unknown', 'sex':'unknown','species':'unknown', 'injury':'not registered', 'fatal_(y/n)':'N/A',
                'investigator_or_source':'unknown', 'pdf':'not applies','href_formula': 'not applies',
                'href': 'not applies', 'unnamed_22': 0, 'unnamed_23': 0}, inplace = True)

In [9]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number     0.02
year            0.03
country         0.79
area            7.22
activity        8.63
age            44.92
time           53.22
dtype: float64

In [10]:
attacks.age = round(pd.to_numeric(attacks.age, errors='coerce'),0) #para convertir todo a float y en caso de no ser float, devuelveme un NAN

In [11]:
attacks.age.fillna(attacks.age.mean(), inplace = True) #para rellenar todos los valores nulos por la media de la columna

In [12]:
attacks.age = attacks.age.astype('int')

In [13]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number     0.02
year            0.03
country         0.79
area            7.22
activity        8.63
time           53.22
dtype: float64

In [14]:
activitymode = attacks.activity.mode()

In [15]:
attacks.activity.fillna('activitymode', inplace=True)

In [16]:
#attacks.case_number.replace(to_replace=r'[a-zA-Z]', value = '', regex=True, inplace=True)
#attacks

#attacks.case_number = attacks.to_datetime(attacks.case_number, format='%Y.%m.%d', errors='coerce')


In [17]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number     0.02
year            0.03
country         0.79
area            7.22
time           53.22
dtype: float64

In [18]:
attacks.case_number=[int(e) for e in range(1, len(attacks)+1)]

In [19]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

year        0.03
country     0.79
area        7.22
time       53.22
dtype: float64

In [20]:
attacks.date = attacks.date.str.replace('Before ', '')

In [21]:
attacks.date[attacks.date.apply(len)!=11] = np.nan

In [22]:
attacks.dropna(subset=['date'], inplace=True)

In [23]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

country     0.31
area        4.19
time       41.92
dtype: float64

In [31]:
attacks.year = attacks.year.astype('int')

In [ ]:
attacks.time.fillna()

In [33]:
#attacks.time[attacks.time.apply(len)!=5] = np.nan

TypeError: object of type 'float' has no len()

In [29]:
attacks.time.replace(to_replace = r'*[a-zA-Z]*', value = '', regex=True, inplace=True)


error: nothing to repeat at position 0

In [27]:
attacks.time.unique()

array(['18h00', '14h00  -15h00', '07h45', nan, 'Late afternoon', '17h00',
       '14h00', 'Morning', '15h00', '08h15', '11h00', '10h30', '10h40',
       '16h50', '07h00', '09h30', 'Afternoon', '21h50', '09h40', '08h00',
       '17h35', '19h00, Dusk', 'Night', '16h00', '15h30', '15h01',
       '12h00', '13h45', '23h30', '09h00', '14h30', '18h30', '12h30',
       '16h30', '18h45', '06h00', '10h00', '10h44', '13h19', 'Midday',
       '13h30', '10h45', '11h20', '11h45', '19h30', '08h30', '15h45',
       'Shortly before 12h00', '17h34', '17h10', '11h15', '08h50',
       '17h45', '13h00', '10h20', '13h20', '07h30', '02h00', '09h50',
       '11h30', '17h30', '9h00', '10h43', 'After noon', '15h15', '15h40',
       '19h05', '1300', '14h30 / 15h30', '22h00', '16h20', '14h34',
       '15h25', '14h55', '17h46', 'Morning ', '15h49', '19h00',
       '09h30 / 10h00', '10h15', '18h15', '04h00', '14h50', '13h50',
       '19h20', '10h25', '10h45-11h15', '16h45', '15h52', '06h15',
       '14h40', '19h45'